# Task 3 model implementation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from model import Model, Optimizer, NetworkModification, TransitScheduleModification

Creating a model implementation only requires pointing to a folder with all the necessary input data. Here, we have a region with 4 microtypes and four modes (walk, bus, auto, and rail), with rail only service two of the four microtypes

Once the model is created, you find supply/demand equilibrium and gather statistics such as mode splits, speeds, and user costs for an individual time period or for the whole day.

In [2]:
model = Model("input-data")

userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

['bus: 0.05214259737806823', 'auto: 0.6260025101042426', 'walk: 0.021604016257977636', 'rail: 0.3002508762597117']
              A          B         C          D
bus    7.583612   6.609924  6.708784   8.137864
auto  11.704070  10.227062  9.006084   9.806835
walk   1.400000   1.400000  1.400000   1.400000
rail  15.000000        NaN       NaN  20.000000
Total user costs = 142723.92991274485


Once a model is created, there are a set of objects and methods that make it easy to modify the transportation system without needing to change the original input files.

Here, we will devote some road space to dedicated bus lanes. In particular, we are transitioning 1000 lane-meters of roadway from mixed bus/car traffic to bus-only lanes in microtypes A and B:

In [3]:
networkModification = NetworkModification(np.array([1000., 1000.]), list(zip([2, 4], [13, 14])))
model.modifyNetworks(networkModification = networkModification)

Once these changes are made, we can re-equilibrate the model and look at how metrics have changed. Here, we have (slightly) increased the bus mode share, in large part by increasing average bus speed in Microtypes A and B. However, because we have decreased auto speeds in these two microtypes, total user costs have gone up.

In [4]:
userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

['walk: 0.02157156306785649', 'auto: 0.6195548581230051', 'bus: 0.058451259752889', 'rail: 0.3004223190562496']
              A         B         C          D
walk   1.400000  1.400000  1.400000   1.400000
auto  11.640533  9.625898  9.082415   9.829095
bus    7.764352  7.281305  6.718737   8.125743
rail  15.000000       NaN       NaN  20.000000
Total user costs = 143492.5730709433


You can also modify the headways for the different transit modes. Here, we can decrease the headway of the bus mode in Microtype A to 200 seconds.

In [5]:
scheduleModification = TransitScheduleModification([200.0], [("A", "bus")])
model.modifyNetworks(scheduleModification = scheduleModification)
userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

['bus: 0.06481848790383433', 'auto: 0.6161895532149635', 'walk: 0.021035071526424462', 'rail: 0.2979568873547777']
              A         B         C          D
bus    7.858933  7.207186  6.674079   8.120797
auto  11.609340  9.660568  9.115994   9.834621
walk   1.400000  1.400000  1.400000   1.400000
rail  15.000000       NaN       NaN  20.000000
Total user costs = 143431.86672395997


Here, we can see that the bus mode share went up (because of lower wait times). In addition, bus speeds in microtype A increased because there was less crowding on each bus, but bus speeds slightly decreased in the other microtypes because of the increase in bus ridership without a corresponding decrease in headways.

## Optimization wrapper

In order to run optimization on the model, we can create an "Optimizer" object that contains a model implementation as well as a direct pathway to modifying model parameters.



In [6]:
networksToReallocate = list(zip([2, 4, 6, 8], [13, 14, 15, 16]))
modesToModify = list(zip(["A", "B", "C", "D"], ["bus", "bus", "bus", "bus"]))
    
o = Optimizer("input-data",fromToSubNetworkIDs= networksToReallocate,
                  modesAndMicrotypes= modesToModify,
                  method="noisy")
default = o.x0()

Done


Once this model is initialized, we can evaluate the total costs in the base, unmodified state

In [7]:
o.evaluate(default)

[ 10.  10.  10.  10. 300. 300. 300. 300.]
132662.04581372117


132662.04581372117

Looking in the optimizer, we can get the state of the transportation system given the input control variable

In [8]:
modeSplit = o.model.getModeSplit('AM-Peak')
speeds = o.model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)

['bus: 0.07154933217316435', 'auto: 0.6087601183546578', 'rail: 0.299733425235247', 'walk: 0.0199571242369304']
              A          B         C          D
bus    7.476487   6.974900  7.029189   8.181209
auto  11.731406  10.316665  9.099145   9.788423
rail  15.000000        NaN       NaN  20.000000
walk   1.400000   1.400000  1.400000   1.400000


We can use this to look at the model performance under a range of inputs. Here we vary headway for buses in microtype A

In [ ]:
    busHeadwayA = np.arange(180, 1260, 60)
    speeds = dict()
    modeSplits = dict()
    totalCost = []
    for hw in busHeadwayA:
        default[4] = hw
        cost = o.evaluate(default)
        totalCost.append(cost)
        speeds[str(hw)] = o.model.getModeSpeeds('AM-Peak').stack()
        modeSplits[str(hw)] = pd.Series(o.model.getModeSplit('AM-Peak').toDict())
        
    speeds = pd.DataFrame(speeds).transpose()
    modeSplits = pd.DataFrame(modeSplits).transpose()
    
    plt.scatter(busHeadwayA, totalCost)
    plt.xlabel("Bus headway in Microtype A (sec)")
    plt.ylabel("Total Cost")

[ 10.  10.  10.  10. 180. 300. 300. 300.]
135096.11310344512
[ 10.  10.  10.  10. 240. 300. 300. 300.]
133312.82413957175
[ 10.  10.  10.  10. 300. 300. 300. 300.]
132596.5832241072
[ 10.  10.  10.  10. 360. 300. 300. 300.]
132357.46000902145
[ 10.  10.  10.  10. 420. 300. 300. 300.]
132363.92828605592
[ 10.  10.  10.  10. 480. 300. 300. 300.]
132503.82450680074


We can plot mode splits and speeds in the different microtypes

In [ ]:
plt.scatter(busHeadwayA, modeSplits["bus"], label="Bus")
plt.scatter(busHeadwayA, modeSplits["auto"], label="Auto")
plt.scatter(busHeadwayA, modeSplits["walk"], label="Walk")
plt.scatter(busHeadwayA, modeSplits["rail"], label="Rail")
plt.xlabel("Bus headway in Microtype A (sec)")
plt.ylabel("Mode split")
plt.legend()

In [ ]:
plt.scatter(busHeadwayA, speeds[("bus", "A")], marker='s', label="Bus speed")
plt.scatter(busHeadwayA, speeds[("auto", "A")], label="Auto speed A")
plt.scatter(busHeadwayA, speeds[("auto", "B")], label="Auto speed B", s=10)
plt.scatter(busHeadwayA, speeds[("auto", "C")], label="Auto speed C", s=10)
plt.scatter(busHeadwayA, speeds[("auto", "D")], label="Auto speed D", s=10)
plt.xlabel("Bus headway in Microtype A (sec)")
plt.ylabel("Speed (m/s)")
plt.legend()

We can also vary 

In [ ]:
    default = o.x0()
    laneDedication = np.arange(0, 1000, 100)
    speeds = dict()
    modeSplits = dict()
    totalCost = []
    for ded in laneDedication:
        default[0] = ded
        cost = o.evaluate(default)
        totalCost.append(cost)
        speeds[str(ded)] = o.model.getModeSpeeds('AM-Peak').stack()
        modeSplits[str(ded)] = pd.Series(o.model.getModeSplit('AM-Peak').toDict())
        
    speeds = pd.DataFrame(speeds).transpose()
    modeSplits = pd.DataFrame(modeSplits).transpose()
    
    
    plt.scatter(laneDedication, totalCost)
    plt.xlabel("Bus dedicated lanes in Microtype A (m)")
    plt.ylabel("Total Cost")

In [ ]:
plt.scatter(laneDedication, speeds[("bus", "A")], marker='s', label="Bus speed")
plt.scatter(laneDedication, speeds[("auto", "A")], label="Auto speed A")
plt.scatter(laneDedication, speeds[("auto", "B")], label="Auto speed B", s=10)
plt.scatter(laneDedication, speeds[("auto", "C")], label="Auto speed C", s=10)
plt.scatter(laneDedication, speeds[("auto", "D")], label="Auto speed D", s=10)
plt.xlabel("Bus dedicated lanes in Microtype A (m)")
plt.ylabel("Speed (m/s)")
plt.legend()